# Trainingspipeline 3

- Zeiträume: 50 Jahre, auf 2.15 Mio Tokens normalisiert
- ohne Bigramme
- vector_size: 300
- window: 10

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_norm.csv', sep=',', encoding='utf-8')

In [4]:
df.shape

(134168, 11)

In [5]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')
df.cleaned_text = df.cleaned_text.fillna('')

In [6]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [7]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1850']
df4 = df_periods['1851-1900']
df5 = df_periods['1901-1950']
df6 = df_periods['1951-2000']
df7 = df_periods['2001-2021']

## Training von Word2Vec

In [8]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [9]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 10                        # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1
negative = 10                      # negative sampling, how many “noise words” should be drawn

### Zeitraum 1: 1700-1750

In [10]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [11]:
%%time
sentences1 = tokenize_text(text1)     # lemmatisierten Text bereinigen und tokenisieren (Ziel: Liste von tokenisierten Sätzen)

Wall time: 12.2 s


In [12]:
print(sentences1[10])

['mettere', 'tasca', 'senza', 'avvedermene', 'viglietto', 'ora', 'rimare', 'perch', 'restare', 'privo', 's', 'bello', 'cosa']


In [13]:
len(sentences1)

82863

In [14]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 3min 34s


In [15]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7870546579360962),
 ('milanesi', 0.7844910025596619),
 ('spaventare', 0.7808846235275269),
 ('goti', 0.7754478454589844),
 ('precipitoso', 0.7691877484321594),
 ('infestare', 0.768860399723053),
 ('tumultuare', 0.7678808569908142),
 ('macello', 0.7676267027854919),
 ('riempiè', 0.7628832459449768),
 ('sconfiggere', 0.7623134851455688)]

In [16]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2Vec3', '3w2v1.model'))

### Zeitraum 2: 1751-1800

In [17]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [18]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 12.9 s


In [19]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 3min 12s


In [20]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('atterrire', 0.892286479473114),
 ('tirannico', 0.8823738098144531),
 ('respingere', 0.8782923221588135),
 ('attentato', 0.8768781423568726),
 ('sgherro', 0.8752859830856323),
 ('sospingere', 0.8724870085716248),
 ('tregua', 0.8691046833992004),
 ('tirannia', 0.8672475814819336),
 ('reprimere', 0.867111325263977),
 ('sconvolgere', 0.8670650720596313)]

In [21]:
w2v2.save(os.path.join('trained_models/Word2Vec3', '3w2v2.model'))

### Zeitraum 3: 1801-1850

In [22]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [23]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 12.4 s


In [24]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 3min 33s


In [25]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.8791089653968811),
 ('morbo', 0.8768715858459473),
 ('furore', 0.876820981502533),
 ('atterrire', 0.8702523708343506),
 ('insidia', 0.8690011501312256),
 ('livor', 0.8649476766586304),
 ('rancore', 0.8644315004348755),
 ('disperazione', 0.8620113730430603),
 ('frenare', 0.8614199757575989),
 ('foga', 0.8603178858757019)]

In [26]:
w2v3.save(os.path.join('trained_models/Word2Vec3', '3w2v3.model'))

### Zeitraum 4: 1851-1900

In [27]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [28]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 12.4 s


In [29]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 2min 52s


In [30]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('parossismo', 0.8530621528625488),
 ('raccapriccio', 0.8291597962379456),
 ('fieramente', 0.8275232911109924),
 ('invasare', 0.8169564604759216),
 ('strazio', 0.8147304058074951),
 ('istantaneo', 0.8103154897689819),
 ('assalire', 0.8099175691604614),
 ('abbattimento', 0.8097150921821594),
 ('convulsione', 0.8090085983276367),
 ('allucinazione', 0.8043323755264282)]

In [31]:
w2v4.save(os.path.join('trained_models/Word2Vec3', '3w2v4.model'))

### Zeitraum 5: 1901-1950

In [32]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [33]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 12.9 s


In [34]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 3min


In [35]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('ira', 0.8522279262542725),
 ('orrore', 0.8500442504882812),
 ('subitaneo', 0.8465667963027954),
 ('raccapriccio', 0.8425236344337463),
 ('spaventevole', 0.840248167514801),
 ('disperazione', 0.8369109630584717),
 ('spavento', 0.8353965878486633),
 ('straziante', 0.8306857943534851),
 ('fremere', 0.8298418521881104),
 ('collera', 0.8254453539848328)]

In [36]:
w2v5.save(os.path.join('trained_models/Word2Vec3', '3w2v5.model'))

### Zeitraum 6: 1951-2000

In [37]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [38]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 19.6 s


In [39]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 3min 18s


In [40]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('purificatore', 0.7975577712059021),
 ('opprimere', 0.7973796725273132),
 ('brutalità', 0.7942426204681396),
 ('sconvolgere', 0.7916878461837769),
 ('preda', 0.7911155223846436),
 ('orribile', 0.7896527051925659),
 ('orrore', 0.7875960469245911),
 ('impotente', 0.7856796979904175),
 ('serba', 0.7841917276382446),
 ('sanguinoso', 0.783898115158081)]

In [41]:
w2v6.save(os.path.join('trained_models/Word2Vec3', '3w2v6.model'))

### Zeitraum 7: 2001-2021

In [42]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [43]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 8.99 s


In [44]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed,
                negative=negative)

Wall time: 4min 6s


In [45]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.8626686334609985),
 ('sgomentare', 0.8312943577766418),
 ('barbaro', 0.8159660696983337),
 ('crudele', 0.808864951133728),
 ('orribilmente', 0.8068526387214661),
 ('islamista', 0.8035518527030945),
 ('raccapriccio', 0.8032824397087097),
 ('tormento', 0.802788496017456),
 ('dèi', 0.802367091178894),
 ('boko', 0.801445722579956)]

In [46]:
w2v7.save(os.path.join('trained_models/Word2Vec3', '3w2v7.model'))